In [1]:
from src.helper_functions import hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from src.symmetric_encryption import is_ecb_mode
from src.xor import xor_bytes
import src.xor as xor
import src.load as load
import src.aes128 as aes128
import src.convert as convert
import src.padding as padding
# 
import base64
import secrets

## **Challenge 17: PKCS#7 padding validation**
# TODO: Implement PKCS#7 padding validation

In [2]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_17_key = secrets.token_bytes(16)

In [3]:
def encryption_oracle_17() -> tuple[bytes, bytes]:
    plaintext_list = [
        b"MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
        b"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
        b"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
        b"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
        b"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
        b"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
        b"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
        b"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
        b"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
        b"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93",
    ]

    chosen_plaintext = secrets.choice(plaintext_list)
    chosen_plaintext = base64.b64decode(chosen_plaintext)
    chosen_plaintext = padding.apply_pkcs_7(chosen_plaintext, aes128.KEY_LENGTH)
    
    IV = secrets.token_bytes(16)
    return aes128.cbc_encrypt(chosen_plaintext, encryption_oracle_17_key, IV), IV


def correct_padding_oracle(ciphertext: bytes, IV: bytes) -> bool:
    plaintext = aes128.cbc_decrypt(ciphertext, encryption_oracle_17_key, IV)
    try:
        padding.remove_pkcs_7(plaintext)
        return True
    except Exception:
        return False

In [4]:
cyphertext, IV = encryption_oracle_17()
correct_padding_oracle(cyphertext, IV)

True

In [5]:
cyphertext_blocks = [IV] + convert.bytes2blocks(cyphertext, aes128.KEY_LENGTH)
cyphertext_blocks

[b'\xac\x9a\xcc\xca\x95\xb83\xe8\xc3H\xf6\xc1\x92\x1d\xc8\xb8',
 b'\xc0\xa8#\x9f[\x15J\xdcC\x11k\xce\x84`!\xb8',
 b'\xb4\x10\\S\x82\xc3@\xc9\x18\xc2+\xa9\xf2\xfc\x17G',
 b'*\xe9jz\x83(/\xf2\x1a\xce\xcb\x1b5\x1b\xfc\xe0']

In [6]:

# # 1 * convert.int2byte(1)
# plaintext_block = aes128.KEY_LENGTH * convert.int2byte(0)

# zeros_IV = aes128.KEY_LENGTH * convert.int2byte(0)

# for i in range(256):
#     plaintext_block = 15 * convert.int2byte(0) + convert.int2byte(i)
#     padding_block = padding.apply_pkcs_7(convert.int2byte(0) * (aes128.KEY_LENGTH - 1), aes128.KEY_LENGTH)
#     fake_IV = xor_bytes(plaintext_block, xor_bytes(padding_block, cyphertext_blocks[1]))

#     zeroing_IV = xor_bytes(padding_block, cyphertext_blocks[1])

#     if correct_padding_oracle(cyphertext_blocks[1], fake_IV):
#         print(i)





## **Challenge 18: Implement CTR, the stream cipher mode**

In [7]:
cyphertext = base64.b64decode("L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==")
key = b"YELLOW SUBMARINE"
plaintext = aes128.ctr_decrypt(cyphertext, key, 0)
print(plaintext.decode())

Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


## **Challenge 19: Break fixed-nonce CTR mode using substitutions**

In [8]:
challenge_19_key = secrets.token_bytes(16)
challenge_19_nonce = 0

In [9]:
# Load list of plaintext strings
plaintext_list = [
    "SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==",
    "Q29taW5nIHdpdGggdml2aWQgZmFjZXM=",
    "RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==",
    "RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=",
    "SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk",
    "T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
    "T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=",
    "UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==",
    "QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=",
    "T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl",
    "VG8gcGxlYXNlIGEgY29tcGFuaW9u",
    "QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==",
    "QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=",
    "QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==",
    "QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=",
    "QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=",
    "VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==",
    "SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==",
    "SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==",
    "VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==",
    "V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==",
    "V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==",
    "U2hlIHJvZGUgdG8gaGFycmllcnM/",
    "VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=",
    "QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=",
    "VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=",
    "V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=",
    "SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==",
    "U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==",
    "U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=",
    "VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==",
    "QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu",
    "SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=",
    "VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs",
    "WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=",
    "SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0",
    "SW4gdGhlIGNhc3VhbCBjb21lZHk7",
    "SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=",
    "VHJhbnNmb3JtZWQgdXR0ZXJseTo=",
    "QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=",
]
plaintext_list = [base64.b64decode(plaintext) for plaintext in plaintext_list]

In [10]:
# Generate the list of cyphertext strings
cyphertext_list = []
for plaintext in plaintext_list:
    cyphertext_list.append(aes128.ctr_encrypt(plaintext, challenge_19_key, challenge_19_nonce))

In [11]:
# We will solve challenge 19 by using the method from challenge 20 (substitution was too hard)
min_len = min([len(cyphertext) for cyphertext in cyphertext_list])
sliced_cyphertext_list = [cyphertext[:min_len] for cyphertext in cyphertext_list]
cyphertext = b''.join(sliced_cyphertext_list)
plaintext, key = xor.repeating_key_break(cyphertext, min_len)
print(plaintext.decode())

i have met them at ccoming with vivid fafrom counter or deskeighteenth-century hi have passed with aor polite meaninglesor have lingered awhpolite meaningless wand thought before Iof a mocking tale orto please a companioaround the fire at tbeing certain that tbut lived where motlall changed, changeda terrible beauty isthat woman's days wein ignorant good wilher nights in argumeuntil her voice grewwhat voice more sweewhen young and beautshe rode to harriersthis man had kept a and rode our winged this other his helpewas coming into his he might have won faso sensitive his natso daring and sweet this other man I hada drunken, vain-glorhe had done most bitto some who are nearyet I number him in he, too, has resignein the casual comedyhe, too, has been chtransformed utterly:a terrible beauty is


## **Challenge 20: Break fixed-nonce CTR statistically**

In [12]:
challenge_20_key = secrets.token_bytes(16)
challenge_20_nonce = 0

In [13]:
# Generate the list of cyphertext strings
cyphertext_list = []
with open('./challenge_data/20.txt', 'r') as f:
    for line in f:
        plaintext = base64.b64decode(line.strip("\n"))
        cyphertext = aes128.ctr_encrypt(plaintext, challenge_20_key, challenge_20_nonce)
        cyphertext_list.append(cyphertext)

In [14]:
min_len = min([len(cyphertext) for cyphertext in cyphertext_list])
sliced_cyphertext_list = [cyphertext[:min_len] for cyphertext in cyphertext_list]
cyphertext = b''.join(sliced_cyphertext_list)
plaintext, key = xor.repeating_key_break(cyphertext, min_len)
print(plaintext.decode())

n'm rated "R"...this is a warning, ya better void / Pduz I came back to attack others in spite- / Strike leut don't be afraid in the dark, in a park / Not a sc~a tremble like a alcoholic, muscles tighten up / Whatuddenly you feel like your in a horror flick / You gjusic's the clue, when I come your warned / Apocalypsoaven't you ever heard of a MC-murderer? / This is thceath wish, so come on, step to this / Hysterical ideariday the thirteenth, walking down Elm Street / You shis is off limits, so your visions are blurry / All serror in the styles, never error-files / Indeed I'm aor those that oppose to be level or next to this / Iporse than a nightmare, you don't have to sleep a winalashbacks interfere, ya start to hear: / The R-A-K-Ishen the beat is hysterical / That makes Eric go get toon the lyrical format is superior / Faces of death jC's decaying, cuz they never stayed / The scene of ashe fiend of a rhyme on the mic that you know / It's jelodies-unmakable, pattern-unescapable / A ho

## **Challenge 21: Implement the MT19937 Mersenne Twister RNG**

In [15]:
def mersenne_twister():
    x = 3
    return -1

mersenne_twister()

-1